# Mouse Population Density Estimation

Here I'm playing around with creating a small neural network to take in the results of the simualtions and try to predict the true population density.

## Notes

I've tried using just the density estimations of each square and it didn't do much better than a linear model (somewhat unsurprisingly). I'm **now incorperating trap spacing and catch radius**. 

## To-Do

- Use some sort of inverse weighting scheme because most of the estimates are already good because the model works well. We need to emphasize the areas where the method doesn't work as well to beat a linear model.
- Try adding in other metrics (variance of the error, for instance) and maybe even make it the target for training rather than L2 error.

In [2]:
import pandas as pd
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxboard import SummaryWriter
import numpy as np

In [3]:
batch_size = 10000
epochs = 10
learning_rate = 0.00001

num_train_samples = 80000
num_test_samples = 20000

context = mx.gpu()

In [4]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(1))
    net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=context)

net

Sequential(
  (0): Dense(None -> 8, Activation(relu))
  (1): Dense(None -> 8, Activation(relu))
  (2): Dense(None -> 8, Activation(relu))
  (3): Dense(None -> 1, linear)
)

In [5]:
simdata = pd.read_csv("data/trainingdata.csv")
simdata.shape

(14128855, 12)

In [6]:
simtrain = simdata.sample(n=num_train_samples)
simtest = simdata.drop(simtrain.index).sample(num_test_samples)

predictors = ["square1", "square2","square3", "square4","square5", "square6","square7", "square8", "TrapSpacing", "CatchRadius"]

Xtrain = nd.array(simtrain[predictors], ctx=context)
Ytrain = nd.array(simtrain["Density"], ctx=context)

Xtest = nd.array(simtest[predictors], ctx=context)
Ytest = nd.array(simtest["Density"], ctx=context)

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtrain, Ytrain), batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtest, Ytest), batch_size=batch_size, shuffle=False)

In [ ]:
num_training_batches = np.ceil(Xtrain.shape[0]/batch_size)
num_training_batches

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})

metric = mx.metric.MSE() # train metric
loss = gluon.loss.L2Loss() # L2 loss

# define a summary writer that logs data and flushes to the file every 5 seconds
sw = SummaryWriter(logdir='./logs', flush_secs=5)

In [ ]:
def test(ctx):
    metric = mx.metric.MSE()
    for data, label in test_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])

    return metric.get()

In [ ]:
# Train the network
global_step = 0
for epoch in range(epochs):
    # reset data iterator and metric at begining of epoch.
    metric.reset()
    
    # training loop for epoch
    for i, (data, label) in enumerate(train_data):
        # Copy data to context (ctx) if necessary
        data = data.as_in_context(context)
        label = label.as_in_context(context)
        # Start recording computation graph with record() section.
        # Recorded graphs can then be differentiated with backward.
        with autograd.record():
            output = net(data)
            L = loss(output, label)
        sw.add_scalar(tag='train_loss', value=L.mean().asscalar(), global_step=global_step)
        global_step += 1
        L.backward()

        # take a gradient step with batch_size equal to data.shape[0]
        trainer.step(data.shape[0])
        
        # update metric at last.
        metric.update([label], [output])
        
    # logging training accuracy
    name, train_acc = metric.get()
    sw.add_scalar(tag='accuracy_curves', value=('train_acc', train_acc), global_step=epoch)
    
    # logging testing accuracy
    name, test_acc = test(context)
    sw.add_scalar(tag='accuracy_curves', value=('valid_acc', test_acc), global_step=epoch)
    
    # Record input and output samples
    # sample 10 random inputs and outputs
    sampleidx = nd.random_randint(low=0, high=Ytest.shape[0], shape=10, ctx=context)
    Xsamples = Xtest[sampleidx]
    output_samples = np.array2string(net(Xsamples)[:,0].asnumpy())
    expected_ouput = np.array2string(Ytest[sampleidx].asnumpy())
    sw.add_text(tag='output_samples', text=input_examples, global_step=epoch)
    sw.add_text(tag='expected_ouput', text=output_examples, global_step=epoch)
    
sw.export_scalars('scalar_dict.json')
sw.close()

In [ ]:
sw.close()

In [8]:
sampleidx = nd.random_randint(low=0, high=Ytest.shape[0], shape=10, ctx=context)
sampleidx


[ 8005  5392 12561 12796 13477 15264 10244 19143  7685  3851]
<NDArray 10 @gpu(0)>

In [11]:
Xtest[sampleidx]


[[ 1.4423077   1.6364645   1.4628534   1.4931583   1.5236686   1.5316814
   1.4989132   1.8356559   2.6         2.4       ]
 [ 1.5873016   4.0137744   2.2893903   1.9476081   2.225613    2.189206
   1.9913498   1.8782991   2.7         0.6       ]
 [ 3.9684072   4.412655    4.3017874   4.2950845   4.326702    4.3838844
   4.419192    4.571713    3.6         1.2       ]
 [ 4.6875      4.296875    2.9513888   2.3481889   3.6920455   5.136268
   6.704623   10.457072    0.4         1.5       ]
 [ 2.366864    1.9600592   2.251808    2.431583    2.4142013   2.43261
   2.7686574   4.0271297   1.3         2.4       ]
 [ 2.5080357   2.3742      2.3921778   2.424567    2.4387534   2.4584908
   2.4211535   2.4465537   4.          1.3       ]
 [ 1.171875    0.87890625  1.0416666   1.1474609   1.34375     1.3780382
   1.5352471   2.0275555   0.8         0.4       ]
 [ 2.4722223   2.5833333   2.425926    2.3975694   2.4177778   2.4583333
   2.4778912   3.009537    3.          2.8       ]
 [ 1.497650

In [12]:
net(Xtest[sampleidx])


[[ -8.623489 ]
 [ 43.810028 ]
 [122.97283  ]
 [ -7.9431996]
 [ 12.049695 ]
 [ 41.803925 ]
 [ 17.071644 ]
 [ 28.930098 ]
 [ 12.234928 ]
 [ 31.174938 ]]
<NDArray 10x1 @gpu(0)>